In [19]:
# Import necessary libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import logging
import operator
from typing import Annotated, TypedDict, Union

from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.prebuilt.tool_executor import ToolExecutor
from langgraph.graph import END, StateGraph, START

In [2]:
# Set up logging
logging.basicConfig(level=logging.INFO)

In [3]:
# Initialize ChatOpenAI model
llm = ChatOpenAI(model="gpt-3.5-turbo", streaming=True)

In [4]:
# Global variables to keep track of the browser
driver = None
page_loaded = False

In [5]:
# Defining tools for the agent to use in the webpage
@tool
def load_page(link: str) -> str:
    """
    Load a webpage using the given link.

    Args:
    link (str): The URL of the webpage to load.

    Returns:
    str: A message indicating success or describing an error if one occurred.
    """
    global driver
    try:
        driver = webdriver.Chrome()
        driver.get(link)
        time.sleep(3)  # Wait for the page to load
        return "Page loaded successfully"
    except Exception as e:
        return f"Error loading webpage: {str(e)}"


@tool
def fill_text_input(label_or_placeholder: str, text: str) -> str:
    """
    Fill a text input field in the form based on the provided label or placeholder text.

    Args:
    label_or_placeholder (str): The label or placeholder text associated with the input field.
    text (str): The text to enter into the input field.

    Returns:
    str: A message indicating success or describing an error if one occurred.
    """
    global driver
    try:
        input_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, f"//div[contains(@data-params, '{label_or_placeholder}')]//input"))
        )
        input_element.send_keys(text)
        return f"Filled input '{label_or_placeholder}' with '{text}'"
    except Exception as e:
        return f"Error filling input '{label_or_placeholder}': {str(e)}"


@tool
def select_radio_or_checkbox(label_text: str) -> str:
    """
    Select a radio button or checkbox in the form based on the provided label text.

    Args:
    label_text (str): The label text associated with the radio button or checkbox.

    Returns:
    str: A message indicating success or describing an error if one occurred.
    """
    global driver
    try:
        element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"//div[contains(@data-params, '{label_text}')]//div[@role='radio' or @role='checkbox']"))
        )
        driver.execute_script("arguments[0].click();", element)
        return f"Selected: {label_text}"
    except Exception as e:
        return f"Error selecting option '{label_text}': {str(e)}"


@tool
def submit_form() -> str:
    """
    Submit the form after it has been filled out.

    Returns:
    str: A message indicating success or describing an error if one occurred.
    """
    global driver
    try:
        submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Submit')]/ancestor::div[contains(@role, 'button')]"))
        )
        submit_button.click()
        return "Form submitted successfully"
    except Exception as e:
        return f"Error submitting form: {str(e)}"


In [6]:
# Define the tools available to the agent
tools = [load_page, fill_text_input, select_radio_or_checkbox, submit_form]

In [7]:
# Create a chat prompt template for the agent
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an assistant helping the user fill out a Google Form. Your task is to extract values from the user's input and use the provided tools to fill out the form. Follow these steps in order:
1. Load the page using the provided link (only do this once at the beginning).
2. Understand the page layout and use flexible matching for input fields (exact match not required).
3. For each piece of information:
   - Use fill_text_input for text fields (name, email, phone number). If exact match fails, use partial match.
   - Use select_radio_or_checkbox for radio buttons and checkboxes.
4. Submit the form when all fields are filled.

The form contains the following fields for your reference
The radio boxes are : Are you a new or existing customer?, How many units would you like to order?
The checkboxes are : What are the item(s) you would like to order?, What color(s) would you like to order?, Preferred contact method
The text fields are : Your name, Phone number, E-mail 

Be sure to match labels flexibly and log potential alternatives if the exact match is not found. Do not load the page more than once.
Use the load_page tool only once."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "Form input details: {input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])


In [8]:
# Create the OpenAI functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [14]:
# Define the agent stat
class AgentState(TypedDict):
    input: str  # The input string from the user
    chat_history: list[BaseMessage]  # List of previous messages in the conversation
    agent_outcome: Union[AgentAction, AgentFinish, None]  # Outcome of the agent call
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]  # Actions and corresponding observations

In [15]:
# It takes in an agent action and calls that tool and returns the result
tool_executor = ToolExecutor(tools)

C:\Users\ACER NITRO\AppData\Local\Temp\ipykernel_16888\2227460658.py:2: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


In [17]:
# Function where the agent in called
def run_agent(data):
    """
    Run the agent with the given data, executing actions based on the user's input.

    Args:
        data (dict): Contains the user's input, chat history, and intermediate steps.

    Returns:
        dict: Contains the agent outcome after invoking the agent.
    """
    inputs = data.copy()
    # Limit the number of intermediate steps retained to the last 5
    if len(inputs["intermediate_steps"]) > 5:
        inputs["intermediate_steps"] = inputs["intermediate_steps"][-5:]
    agent_outcome = agent_runnable.invoke(inputs)  # Invoke the agent with the input data
    return {"agent_outcome": agent_outcome}

In [18]:
def execute_tools(data):
    """Execute the tools based on the agent's action."""
    agent_action = data["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}

def should_continue(data):
    """Determine whether to continue the agent loop or exit."""
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    else:
        return "continue"

In [20]:
# Define the workflow graph
workflow = StateGraph(AgentState)

# Add nodes to the graph
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entry point
workflow.add_edge(START, "agent")

# Add conditional edges
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)

# Add edge from action to agent
workflow.add_edge("action", "agent")

In [21]:
# Compile the workflow
app = workflow.compile()

In [22]:
# Define input for the workflow
input = "Hey, I am an existing customer, and I want to order pens and notebooks of red and blue colors. Quantity of the items should be 4. As per details about me, I'm Kavan, and I'm available at 9548565487 / kavan@gmail.com. Preferred mode of communication is either phone or email. Thanks! \n Forms link : https://docs.google.com/forms/d/e/1FAIpQLSd9gli7KqnYFNkrc_PWNxvmhi7ZJz2jPp0qTsceqT7lkIBo2Q/viewform"
inputs = {
    "chat_history": [],
    "input": input, 
}

In [23]:
# Stream the workflow execution
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'agent_outcome': AgentActionMessageLog(tool='load_page', tool_input={'link': 'https://docs.google.com/forms/d/e/1FAIpQLSd9gli7KqnYFNkrc_PWNxvmhi7ZJz2jPp0qTsceqT7lkIBo2Q/viewform'}, log="\nInvoking: `load_page` with `{'link': 'https://docs.google.com/forms/d/e/1FAIpQLSd9gli7KqnYFNkrc_PWNxvmhi7ZJz2jPp0qTsceqT7lkIBo2Q/viewform'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"link":"https://docs.google.com/forms/d/e/1FAIpQLSd9gli7KqnYFNkrc_PWNxvmhi7ZJz2jPp0qTsceqT7lkIBo2Q/viewform"}', 'name': 'load_page'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-00efbb24-38eb-4b57-ba85-06e0d97ce68e-0')])}
----
{'intermediate_steps': [(AgentActionMessageLog(tool='load_page', tool_input={'link': 'https://docs.google.com/forms/d/e/1FAIpQLSd9gli7KqnYFNkrc_PWNxvmhi7ZJz2jPp0qTsceqT7lkIBo2Q/viewform'}, log="\nInvoking: `load_page` with `{'link': 'https://docs.google.com/forms/d/e/1FAIpQLSd9gli7KqnYFNkrc_P

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'agent_outcome': AgentActionMessageLog(tool='select_radio_or_checkbox', tool_input={'label_text': 'Existing customer'}, log="\nInvoking: `select_radio_or_checkbox` with `{'label_text': 'Existing customer'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"label_text":"Existing customer"}', 'name': 'select_radio_or_checkbox'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-209d37dd-87ab-4b58-9eea-4a37a21f6fb1-0')])}
----
{'intermediate_steps': [(AgentActionMessageLog(tool='select_radio_or_checkbox', tool_input={'label_text': 'Existing customer'}, log="\nInvoking: `select_radio_or_checkbox` with `{'label_text': 'Existing customer'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"label_text":"Existing customer"}', 'name': 'select_radio_or_checkbox'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'agent_outcome': AgentActionMessageLog(tool='select_radio_or_checkbox', tool_input={'label_text': 'red'}, log="\nInvoking: `select_radio_or_checkbox` with `{'label_text': 'red'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"label_text":"red"}', 'name': 'select_radio_or_checkbox'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-f227b35a-aa25-49d1-9173-fbba02cf15d2-0')])}
----
{'intermediate_steps': [(AgentActionMessageLog(tool='select_radio_or_checkbox', tool_input={'label_text': 'red'}, log="\nInvoking: `select_radio_or_checkbox` with `{'label_text': 'red'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"label_text":"red"}', 'name': 'select_radio_or_checkbox'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-f227b35a-aa25-49d1-9173-fbba02cf15d2-0')]), 'Selected: red')]}
----


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'agent_outcome': AgentActionMessageLog(tool='select_radio_or_checkbox', tool_input={'label_text': 'blue'}, log="\nInvoking: `select_radio_or_checkbox` with `{'label_text': 'blue'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"label_text":"blue"}', 'name': 'select_radio_or_checkbox'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-725fc53e-559a-4eab-a9d0-eef05caac280-0')])}
----


GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.